In [ ]:
import pandas as pd
from IPython.core.debugger import set_trace
import datetime
import requests
import time
from tqdm.notebook import tqdm

In [ ]:
english_names = pd.read_csv('data/articles.csv', header=None)
english_names.columns = ['Title']
english_names['Title'] = english_names['Title'].apply(lambda x : x.replace(' ', '_'))
english_names

In [ ]:
def get_qid_from_title(title, language):
    response = requests.get(f'https://{language}.wikipedia.org/w/api.php?'
                            f'action=query&prop=pageprops&titles={title}&redirects&format=json')
    try:
        r = [item for item in response.json()['query']['pages'].values()][0]
        qid = r['pageprops']['wikibase_item']
    except KeyError:
        print(f'Article {title} has no Wikidata ID')
        return None
    return qid

### Retrieve wikidata entries for each given article

In [ ]:
qids = pd.DataFrame([get_qid_from_title(title, 'en') for title in tqdm(english_names.Title.values)])
qids.head()

### English pageview data

In [ ]:
def day_year_to_date(year, days):
    return datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)

def get_wikishark_id(article_name, lang):
    response = requests.get(f'https://www.wikishark.com/autocomplete.php?q={article_name}')
    r = response.json()
    
    target = None
    for candidate in r:
        if '(' + lang + ')' in candidate['name']:
            target = candidate['id']
    return target

def get_daily_pageviews(titles, language, start, end):
    data = []
    for title in tqdm(titles):
        
        # authors requested we wait 1 second at least in between requests
        time.sleep(1)
        
        wikishark_id = get_wikishark_id(title, language)
        
        # wait to avoid overloading servers
        time.sleep(1)
        
        response = requests.get(f'https://www.wikishark.com/getdata/daily.php?value={wikishark_id}?view=2&scale=0&normalized=0&loglog=0&log=0&zerofix=0')
        daily_data = response.json()
        
        # add data with timestamps
        start_date = datetime.datetime.strptime(start, '%d/%m/%Y')
        end_date   = datetime.datetime.strptime(end, '%d/%m/%Y')
        current_date = datetime.datetime.now()
        
        # wikishark returns daily page views for every day since 2007-12-31 (independent of given parameters)
        # we need to index it according to the time period we are interested in
        start_index = (len(daily_data) - 1) - (current_date - start_date).days
        end_index = (len(daily_data) - 1) - (current_date - end_date).days
        timestamps = {}
        for i, d in enumerate(daily_data[start_index:end_index]):
            ts = start_date + datetime.timedelta(days=i)
            timestamps[ts] = int(d)
    
        data.append({**{'Article': title}, **timestamps})
    
    return pd.DataFrame(data)

In [ ]:
privacy_en = get_daily_pageviews(list(english_names.head(30).Title), 'en', [2011, 2012, 2013,2014])

# use article name as index
privacy_en.index = privacy_en.Article
privacy_en = privacy_en.drop(['Article'], axis=1)
privacy_en.head()

### Aggregate into monthly data

In [ ]:
privacy_en.columns = pd.to_datetime(privacy_en.columns)

# take monthly cumulative
monthly_en = privacy_en.resample('M', axis=1).sum()
monthly_en.head()

# Exploratory data analysis 

In [ ]:
#Melt dataframe to use dates as entry values rather than columns
monthly_en_melt = pd.melt(monthly_en, value_name='views', var_name='date', ignore_index=False)

### Plotting the monthly total pageviews

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot, dates
import datetime

reveal = datetime.datetime(2013, 6, 5) #Around june 
fig, ax = plt.subplots()
ax.axvline(reveal,c='r')
monthly_en_melt.reset_index().groupby('date').sum().reset_index().plot.scatter(x='date',y='views',ax=ax,rot=90)

### Plotting per article views (total)

In [ ]:
monthly_en_melt.reset_index().groupby('Article')['views'].sum().sort_values(ascending=False).plot.bar()

It looks like most of the traffic generated on these pages comes from the 4Chan article, which has been famous for many scandals unrelated to internet privacy, which means also probably a high variance in views. Since we can safely assume this article to not be helpful regarding the tendency of people to look for privacy-enhancing tools, we blacklist this article from our study. Just to be safe : we check the views per months per article.

### Plotting per article views (monthly)

In [ ]:
fig, axs = plt.subplots(6,5,figsize=(20,20),sharey=True)
for article,ax in zip(list(monthly_en_melt.index.unique()),axs.flat):
    monthly_en_melt.loc[article][['date','views']].reset_index().plot.scatter(x='date',y='views',ax=ax,rot=90)
    ax.axvline(reveal,c='r')
    ax.set_title(article)
plt.tight_layout()

Indeed, we decide to blacklist 4Chan from our articles. On another note, even at this scale, we see that there seems to be an immediate impact on the pageviews for DuckDuckGo.

### Removing 4chan from our articles

In [ ]:
monthly_en_melt = monthly_en_melt.drop('4chan')

### A world without 4Chan

In [ ]:
fig, ax = plt.subplots()
ax.axvline(reveal,c='r')
monthly_en_melt.reset_index().groupby('date').sum().reset_index().plot.scatter(x='date',y='views',ax=ax,rot=90)

In [ ]:
fig, axs = plt.subplots(6,5,figsize=(20,20),sharey=True)
for article,ax in zip(list(monthly_en_melt.index.unique()),axs.flat):
    monthly_en_melt.loc[article][['date','views']].reset_index().plot.scatter(x='date',y='views',ax=ax,rot=90)
    ax.axvline(reveal,c='r')
    ax.set_title(article)
plt.tight_layout()

In [ ]:
monthly_en_melt.reset_index().groupby('Article')['views'].sum().sort_values(ascending=False).plot.bar()